In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Columbia EECS E6893 Big Data Analytics
"""
This module is used to pull data from twitter API and send data to
Spark Streaming process using socket. It acts like a client of
twitter API and a server of spark streaming. It open a listening TCP
server socket, and listen to any connection from TCP client. After
a connection established, it send streaming data to it.


Usage:
  If used with dataproc:
    gcloud dataproc jobs submit pyspark --cluster <Cluster Name> twitterHTTPClient.py

  Make sure that you run this module before you run spark streaming process.
  Please remember stop the job on dataproc if you no longer want to stream data.

Todo:
  1. change the credentials to your own

"""

import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
import socket
import re
import json
import csv
import os

# credentials
# TODO: replace with your own credentials
BEARER_TOKEN = '' # your bearer token

# the tags to track
tags = ['#', 'bigdata', 'spark', 'ai', 'movie']
    
class MyStream(tweepy.StreamingClient):

    global client_socket
    def on_tweet(self, tweet):
        try:
            msg = tweet
            print('TEXT:{}\n'.format(msg.text))
            client_socket.send( msg.text.encode('utf-8') )
            sentence = re.sub("[^\u0000-\u05C0\u2100-\u214F]+", ' ', msg.text)
            sentence = re.sub(r"http\S+", ' ', sentence)
            with open('stream_data.csv', 'a', encoding='UTF-8') as f:
                csv_writer = csv.writer(f)
                csv_writer.writerow([sentence])
            client_socket.send( msg.text.encode('utf-8') )    
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            self.disconnect()
            return False
        
    def on_error(self, status):
        print(status)
        return False
        
def sendData(c_socket, tags):
    """
    send data to socket
    """
    global client_socket
    client_socket = c_socket
    stream = MyStream(BEARER_TOKEN)

    for tag in tags:
        stream.add_rules(tweepy.StreamRule(tag))

    stream.filter()


class twitter_client:
    def __init__(self, TCP_IP, TCP_PORT):
      self.s = s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      self.s.bind((TCP_IP, TCP_PORT))

    def run_client(self, tags):
      try:
        self.s.listen(1)
        while True:
          print("Waiting for TCP connection...")
          conn, addr = self.s.accept()
          print("Connected... Starting getting tweets.")
          sendData(conn,tags)
          conn.close()
      except KeyboardInterrupt:
        exit


if __name__ == '__main__':
    if os.path.exists('stream_data.csv'):
        os.remove('stream_data.csv')
    with open('stream_data.csv','a') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(["sentence"])
    client = twitter_client("localhost", 9001)
    client.run_client(tags)

Waiting for TCP connection...
Connected... Starting getting tweets.
TEXT:モイ！iPhoneから #ツイキャスゲームズ を開始しました プレイルームをプレイ #ツイキャスゲームズ
https://t.co/8cztSGlooj

TEXT:RT @WandasAttorney: Black Adam should’ve been a Justice Society movie ft. Black Adam not the other way around… https://t.co/MQ90Dv0Lzt

TEXT:RT @T_Az38: AIを操る者が新たに生み出してしまった「爆乳機関車」とかいう業の深いジャンル、このために生み出された造語かと思いきや、元々特定の形式の機関車を指す言葉として存在してた上に、その言葉の由来がハッキリ分かってないの面白すぎる（

TEXT:RT @T_Az38: AIを操る者が新たに生み出してしまった「爆乳機関車」とかいう業の深いジャンル、このために生み出された造語かと思いきや、元々特定の形式の機関車を指す言葉として存在してた上に、その言葉の由来がハッキリ分かってないの面白すぎる（

TEXT:宇宙
#ポケモン
https://t.co/JWiv59j6lq

TEXT:@rurururu_ai
世の中の値上げに負けてたまるか！キャンペーンにご応募ありがとうございます！

当落結果は…!!

残念…

10月31日まで毎日応募出来るので、明日もご応募お待ちしております。
店頭キャンペーン詳細はコチラ
→https://t.co/yAisUQI6aI https://t.co/6gWUHxSRwm

TEXT:acusado de várias coisas e perdeu a bolsa de monitor. Hj em dia faz med e pagava de pobre,  mas esse ai doeu dms, pq eu considerava um irmão. Não entendo pq eu nunca dou certo com amizade na parte de homens, sabendo q eu trato todos

TEXT:é cd coisa q vejo nesse twitter q daqui a pouco viu... ai ai falo nada

TEXT:RT @buitengebieden: Disney movie in real life.. 😊 https://t.co/2LwGGD2GO1

TEXT:RT @anhtu9s: Bot răm re^n dữ dội vì anh top n ắc quá là sướng. Ai muốn tham gia cùng hông nào? https://t.co/7xY0yxOhKK

TEXT:RT @primalpoly: Highly recommend the epic new 8-hour interview with @balajis on the @lexfridman podcast. 

Topics include the network state…

TEXT:RT @nalltama: 【悲報】AI絵師、『爆乳機関車』という概念を爆誕させる🥶  
https://t.co/6M6LWZBXV4
トーマスじゃん https://t.co/pC2B5sighP

TEXT:お悩み相談…始まったよ②
どんどんかこたえるからきてね
#にしきくん24時間配信
#にしきくん
https://t.co/nWdzZK239x

TEXT:3&gt;Movie was so interesting that in interval I bought 470rs of popcorn n cold drink (while ticket costed 235 rs) which I never ever bought in past. Made myself clear to enjoy the movie at maximum. 
#KantaraTheLegend

TEXT:RT @ai_otsuka99: 開場してます！
本日最終、17時までです、
大阪市中央公会堂にて
ぜひ遊びにきてください🙋‍♀️🙋‍♀️🙋‍♀️

AIO https://t.co/1I02pCSkAW

TEXT:Headspace Health is focused on creating an integ

TEXT:RT @WarpsiwaMovie: Chiharu Ito
Full Movie https://t.co/SxYav366Bc

จบปัญหาแข็งตัวยาก!
ด้วยอาหารเสริม ぺ二活サプリHARD
ผลิตและนำเข้าจากญี่ปุ่น
สูต…

TEXT:Quer me beijar e não responde story e nem minhas fotos??ai complica né gatoo

TEXT:RT @root_nico: ちょっとアレの話してもいいですか。。。。。。
#るぅとくん
https://t.co/d0FHloJfMa

TEXT:アユム氏が昨日の３七桂（アベマAI読み筋）を解説してくれてる。

TEXT:@ai_takanashi またゲーム配信してほしい〜

TEXT:Yes Ted , we can send our kids to school without worrying if they'll be shot ! We can go shopping without worrying we'll be shot , see a movie without worrying we'll be shot , ya Ted it's a terrible way to live !! #TedisownedbytheNRA https://t.co/Q6bWmLkyak

TEXT:Upon seeing pics of me the AI says:
a woman standing in front of a door wearing a brown suit, tumblr contest winner, private press, androgynous, masculine, a woodcut by Grace English, feminist art, studio portrait, woodcut, maximalist, a cubist painting, fauvism, made of trash

TEXT:RT @ACOTeamHonnavar: 4Year For Block Buster #TheVillian Movie 
The fil

TEXT:Jk pls watch it so I can talk about it with someone

TEXT:RT @mocchiri_green: まだAIで遊んでるんですけど、ふわふわ大きい鳥を要求したら遂に望んでたのが来たので報告！ https://t.co/q4OxIafGMt

TEXT:Je vous préviens j'ai plus un kopeck, je vais voir si je peux rapidement me faire de l'argent sur le marché du travail avec mes capacités physiques et mentales, quelqu'un me propose de m'aider à être recrutée #réseau. Je vous dirai combien j'ai touché.

TEXT:RT @ParlayScience: Week 7 NFL Cheat Sheet is ready to go! Notable players that have hit their lines in last 4/3/2 games! As always brought…

TEXT:RT @R2beh: After serye,sunod nmn ang Movie na aabangan natin,kaya ipon ipon na heheh..

LOINIE KILIG TREATS
@iamr2alonte @iamAndalioLoisa

TEXT:RT @Trungnien86: Chim hơi đen
Có ai muốn ngậm thử không nhỉ
Nhìn con chim thích ghê https://t.co/lXNqBD8nsk

TEXT:encore une nuit ou j’ai pas dormis 👍🏻

TEXT:@crittermom77 Classic

TEXT:@cckeipop ai yaparen Kon este sufrimiento

TEXT:RT @MegaFamily_Fans: Get Ready For The Exclusive Space On  

TEXT:RT @AImarikeiba: 新潟9R中穴◎だよ！
面白い指数だったのでnoteで出しました😊

新潟9R 柏崎特別｜競馬AIウマリ @AImarikeiba #note https://t.co/ZfvR2Y0lFv

TEXT:RT @RockybhaiOffcl: .@Suriya_offl's last Diwali release &amp; First 100cr Movie 

Festival Release &gt;&gt; https://t.co/MRZMDYJEQo

TEXT:@jgrant24 i’m gonna read them. i have the first one on my bookshelf. douglas adams was recommended by another friend as well

TEXT:Chris Pratt is playing Superman in the new movie!

TEXT:vindo ai https://t.co/UQ6Oudmz1H

TEXT:Trying to end a long debate! Is Die Hard a Christmas movie or not!? Hit those votes!

TEXT:RT @livedoornews: 【爆誕】監視カメラでAIが認識しないアグリーセーター開発 米
https://t.co/p7lItLswkc

市場のカボチャのような模様は、機械学習システムが認証時に用いるスコアを下げる、脆弱性を調べるために生成された敵対的パターン。「ダサ…

TEXT:RT @WarpsiwaAV: Kana Momonogi
Full Movie https://t.co/MVgSBNSdo3

‼️SLOTXO369‼️
🌟เจ้าใหญ่ เปิดมานาน มั่นใจได้💯
🌟เว็บตรง ได้เท่าไหร่ก็จ่าย โ…

TEXT:RT @animateonline: 🔶土岐隼一さん特集🔶

『東京リベンジャーズ』の羽宮一虎役や、『プロジェクトセカイ』の神代類役など、人気作品のキャラクターを多く演じている #土岐隼一 さんの特集ページです✨
https://t.co/zvwkhdgj

...

TEXT:RT @kiyoe_sans: La boda 
la jugada de la quintis.

#五等分の花嫁 
MANGA color / movie https://t.co/Ey5KQw1IIG

TEXT:RT @aya_misa1126: 阪神9R✨自信度A +✨

🍀RTフォローで全員無料で読めます🍀

AIが導き出した期待値高い馬で勝負🔥

⚠️ 今週絶好調の最強AI🔥

スプリンター◎20.0倍2着✨
凱旋門◎10.3倍2着

🌱前回の中央🌱
東京3R◎8.2倍2着◎▲△…

TEXT:@Megan6663 What an amazing movie.

TEXT:RT @w1nsujuday6: me watching #20thCenturyGirl :

first hour                             the last
&amp; half                                   3…

TEXT:@gauty2166 Hóng! Tới luôn cô ơi :)

TEXT:RT @Colon56N: すとぷり２４時間リレー生放送！さとみころんによる大人の質問コーナー！！！しゅうごおおおおおお！！！
https://t.co/3wkmunjnn6

TEXT:@Sharito742 Loved him!

TEXT:@yetz1 Tropa de elite 3 - a volta dos que não foram

TEXT:ai q deplessao eu poderia ta na festa agr https://t.co/chhacGnDtZ

TEXT:@ManIMissThe90s I wouldn’t say all of them are. Like, some people genuinely enjoyed this movie, while others didn’t. It depends on your taste really.

TEXT:@scrawlingkay Overall great job on this!

TEXT:@nu_ko_ja LINEで「ばりぐっどくん」っていうお絵描きAIを友達検索で探すとでてきますよ☺

TEXT:RT @ouma_ai: はじめまして

6月に撮影会を卒業させていただきました
19歳の✿桜満あい✿です

桜に満天で  おうま  と読みます🌸
165cmの小さな巨人
童顔なむちむち系です
今読んでる本は三日間の幸福です
甘いものと紅茶が好きです🫖

被写体として成長できる…

TEXT:RT @nalltama: 【悲報】AI絵師、『爆乳機関車』という概念を爆誕させる🥶  
https://t.co/6M6LWZBXV4
トーマスじゃん https://t.co/pC2B5sighP

TEXT:ニッチな性癖はヒト絵師じゃ無いと描けないんじゃなかったの…AIくん…

TEXT:@turnintoabat VHS 99 was pretty fun. I really can’t recommend anything I’ve watched recently other than that.

TEXT:RT @Jameca2011: Idc how grown I am, Ima watch me a Disney Movie 😂🤷🏽‍♀️

TEXT:Giờ chỉ thích mb hạng nặng và nhìn vợ bị trai lạ https://t.co/xrYWIVPVI9 có sở thích kì lạ như mình ko ta https://t.co/8z5ORxR7pt

TEXT:@yetz1 Terrifier 2

TEXT:@giabao2k6 Xin slot

TEXT:RT @kanichan_2255: 緊急集合！！！あの伝説のちょこらびアモンガスが帰ってきた！！！！！！！！！！！！！！嘘つきを見破ります！！！！
#ちょこらび
https://t.co/TljGbby2So

TEXT:@AB1SITA no s una enfermita d por ai

TEXT:RT @BunnyNikhil214: Happy Birthday Our Darling #Prabhas ❤️🫂

All The Best For Your Future Projects. Eagerly Waiting For Your Massiest Movie…

TEXT:RT @_kinotake_: 凸待

TEXT:RT @byjaiOG6aoTd5Qj: ⭐️AI搭載のEA代理店募集⭐️

自動売買ツールを一緒に広めてくれるアフィリエイターを募集しています！

🔥月収100万円以上可
🔥初期費用一切なし
🔥ノルマなし
🔥やればやるだけ稼げる

詳細の確認についてはプロフURLからお願いし…

TEXT:$GS near ATM options analytics 

Expiry Date: 28 October 2022

Last Traded Price: $325.1

ATM Call-Put Ratio: 10.73 

$330.0 strike received highest open interest with a call-put ratio of 25.17 

#options $SPY 

Check out our tool at: 
https://t.co/LAvUlvARE9 https://t.co/dN0w8I9CiW

TEXT:RT @laserlime: another question from @arcadekore was what movies the checkmate family watch, so here’s the suggestions each would bring to…

TEXT:RT @livedoornews: 【爆誕】監視カメラでAIが認識しないアグリーセーター開発 米
https://t.co/p7lItLswkc

市場のカボチャのような模様は、機械学習システムが認証時に用いるスコアを下げる、脆弱性を調べるために生成された敵対的パターン。「ダサ…

TEXT:@wargonm @ordre_medecins @PresidentCnom @nathanpsmad Fermez la wargon Nazi. We ré waiting for you.

TEXT:RT @livedoornews: 【爆誕】監視カメラでAIが認識しないアグリーセーター開発 米
https://t.co/p7lItLswkc

市場のカボチャのような模様は、機械学習システムが認証時に用いるスコアを下げる、脆弱性を調べるために生成された敵対的パターン。「ダサ…

TEXT:RT @ouma_ai: 

Stream connection closed by Twitter


TEXT:RT @TangTan08793028: AI의 끝

라면 먹는 라면 https://t.co/ntwq0ycPF9

Error on_data: [Errno 32] Broken pipe
Waiting for TCP connection...
